# Imports

In [14]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt


# General variables

 ## Filenames

In [9]:
# Define the pattern of filenames

GameTheoretic_filename_pattern_DQN =  re.compile(r"results_(?P<simulation_index>\d{3})_(?P<episodes>\d+)_DQN_"
                                                r"(?P<emotion>[^_]+)_(?P<see_emotions>[^_]+)_"
                                                r"(?P<alpha>[\d.]+)_(?P<beta>[\d.]+)_(?P<smoothing>[\d.]+)_(?P<threshold>[\d.]+)_(?P<rounder>[\d.]+)_"
                                                r"(?P<learning_rate>[\d.]+)_(?P<gamma>[\d.]+)_(?P<epsilon>[\d.]+)_(?P<epsilon_decay>[\d.]+)_(?P<epsilon_min>[\d.]+)_"
                                                r"(?P<batch_size>[\d.]+)_(?P<hidden_size>[\d.]+)_(?P<update_target_every>[\d.]+)_"
                                                r"(?P<random_suffix>\d{6})_(?P<suffix>[^_]+)\.csv"
)

GameTheoretic_filename_pattern_QL = re.compile(r"results_(?P<simulation_index>\d{3})_(?P<episodes>\d+)_QLearning_"
                                              r"(?P<emotion>[^_]+)_(?P<see_emotions>[^_]+)_"
                                              r"(?P<alpha>[\d.]+)_(?P<beta>[\d.]+)_(?P<smoothing>[\d.]+)_(?P<threshold>[\d.]+)_(?P<rounder>[\d.]+)_"
                                              r"(?P<learning_rate>[\d.]+)_(?P<gamma>[\d.]+)_(?P<epsilon>[\d.]+)_(?P<epsilon_decay>[\d.]+)_(?P<epsilon_min>[\d.]+)_"
                                              r"(?P<random_suffix>\d{6})_(?P<suffix>[^_]+)\.csv"
)


Maze2D_filename_order_QL = re.compile(
    r"maze2d_results_(?P<simulation_index>\d{3})_(?P<episodes>\d+)_QLearning_"
    r"(?P<emotion>[^_]+)_(?P<see_emotions>[^_]+)_"
    r"(?P<alpha>[\d.]+)_(?P<beta>[\d.]+)_(?P<smoothing>[\d.]+)_(?P<threshold>[\d.]+)_(?P<rounder>[\d.]+)_"
    r"(?P<learning_rate>[\d.]+)_(?P<gamma>[\d.]+)_(?P<epsilon>[\d.]+)_(?P<epsilon_decay>[\d.]+)_(?P<epsilon_min>[\d.]+)_"
    r"(?P<random_suffix>\d{6})_(?P<suffix>[^_]+)\.csv"
)

Maze2D_filename_order_DQN = re.compile(
    r"maze2d_results_(?P<simulation_index>\d{3})_(?P<episodes>\d+)_DQN_"
    r"(?P<emotion>[^_]+)_(?P<see_emotions>[^_]+)_"
    r"(?P<alpha>[\d.]+)_(?P<beta>[\d.]+)_(?P<smoothing>[\d.]+)_(?P<threshold>[\d.]+)_(?P<rounder>[\d.]+)_"
    r"(?P<learning_rate>[\d.]+)_(?P<gamma>[\d.]+)_(?P<epsilon>[\d.]+)_(?P<epsilon_decay>[\d.]+)_(?P<epsilon_min>[\d.]+)_"
    r"(?P<batch_size>[\d.]+)_(?P<hidden_size>[\d.]+)_(?P<update_target_every>[\d.]+)_"
    r"(?P<random_suffix>\d{6})_(?P<suffix>[^_]+)\.csv"
)

FILENAME_PATTERNS = [
    GameTheoretic_filename_pattern_DQN,
    GameTheoretic_filename_pattern_QL,
    Maze2D_filename_order_DQN,
    Maze2D_filename_order_QL
]

FILENAME_PATTERNS_PAIR = [
    ("Gametheoretic", GameTheoretic_filename_pattern_DQN),
    ("Gametheoretic", GameTheoretic_filename_pattern_QL),
    ("maze2d", Maze2D_filename_order_DQN),
    ("maze2d", Maze2D_filename_order_QL)
]

# Functions

## CSV processing

### Parameter recovery from filenames

In [10]:
def parse_results_filenames(folder_path: str) -> pd.DataFrame:
    """
    Scans a folder for result filenames and extracts simulation parameters into a DataFrame.

    Args:
        folder_path (str): Path to the folder containing result CSV files.

    Returns:
        pd.DataFrame: DataFrame containing parsed parameters from filenames.
    """
    data = []

    for filename in os.listdir(folder_path):
        if not filename.endswith(".csv"):
            continue

        for pattern in FILENAME_PATTERNS:
            match = pattern.match(filename)
            if match:
                file_data = match.groupdict()
                file_data["filename"] = filename
                data.append(file_data)
                break  # Stop at the first match

    if not data:
        print("No matching filenames found.")
        return pd.DataFrame()

    df = pd.DataFrame(data)

    # Optional: convert numeric fields from str to float/int
    for col in df.columns:
        if col not in {"filename", "emotion", "see_emotions", "suffix"}:
            try:
                df[col] = pd.to_numeric(df[col])
            except:
                pass  # leave as string if conversion fails

    return df

In [11]:
def aggregate_results_by_suffix(folder_path: str, target_suffix: str, source_filter: str = None) -> pd.DataFrame:
    """
    Aggregates CSV files with a given suffix and optionally filters by source ('maze2d' or 'gametheoretic').

    Args:
        folder_path (str): Folder containing result files.
        target_suffix (str): e.g. "episode_summary" or "step_data".
        source_filter (str, optional): If set to "maze2d" or "gametheoretic", only those files will be included.

    Returns:
        pd.DataFrame: Aggregated DataFrame with data and filename metadata.
    """
    all_data = []

    for filename in os.listdir(folder_path):
        if not filename.endswith(".csv"):
            continue

        for source_type, pattern in FILENAME_PATTERNS:
            if source_filter and source_type != source_filter:
                continue  # Skip if not matching the desired source

            match = pattern.match(filename)
            if match:
                metadata = match.groupdict()
                if metadata.get("suffix") == target_suffix:
                    file_path = os.path.join(folder_path, filename)
                    try:
                        df = pd.read_csv(file_path)
                        for key, value in metadata.items():
                            df[key] = value
                        df["source"] = source_type
                        all_data.append(df)
                    except Exception as e:
                        print(f"Error reading {filename}: {e}")
                break  # Stop at first matching pattern

    if not all_data:
        print(f"No matching files found for suffix '{target_suffix}' and source '{source_filter}'.")
        return pd.DataFrame()

    final_df = pd.concat(all_data, ignore_index=True)

    # Try numeric conversion for non-categorical columns
    for col in final_df.columns:
        if col not in {"emotion", "see_emotions", "suffix", "filename", "source"}:
            try:
                final_df[col] = pd.to_numeric(final_df[col])
            except:
                pass

    # Save output
    filtered_tag = f"_{source_filter}" if source_filter else ""
    output_filename = f"aggregated_{target_suffix}{filtered_tag}.csv"
    output_path = os.path.join(folder_path, output_filename)
    final_df.to_csv(output_path, index=False)
    print(f"Saved aggregated data to: {output_path}")

    return final_df

### Raw files to summary DataFrame and csv

## Data analysis

 ### Learning verification

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def windowed_avg_combined_reward(
    df: pd.DataFrame,
    reward_prefix: str = "total_combined_reward_",
    episode_column: str = "episode",
    simulation_id_column: str = "simulation_index",
    window_size: int = 5,
    aggregation_mode: str = "mean",  # or "best"
    plot: bool = False
) -> pd.DataFrame:
    """
    Computes a windowed moving average of combined rewards per episode across simulations.

    Args:
        df (pd.DataFrame): Input dataframe.
        reward_prefix (str): Prefix of reward columns per agent.
        episode_column (str): Column name for episodes.
        simulation_id_column (str): Column indicating different simulations.
        window_size (int): Window size for moving average.
        aggregation_mode (str): 'mean' for average across agents, 'best' for max reward among agents.
        plot (bool): Whether to plot the result.

    Returns:
        pd.DataFrame: DataFrame with ['episode', 'aggregated_reward', 'moving_avg'].
    """
    reward_cols = [col for col in df.columns if col.startswith(reward_prefix)]
    if not reward_cols:
        raise ValueError(f"No columns found with prefix '{reward_prefix}'")

    if aggregation_mode == "mean":
        df["aggregated_reward"] = df[reward_cols].mean(axis=1)
    elif aggregation_mode == "best":
        df["aggregated_reward"] = df[reward_cols].max(axis=1)
    else:
        raise ValueError("aggregation_mode must be 'mean' or 'best'")

    # Group by episode and average over simulations
    episode_avg = (
        df.groupby(episode_column)["aggregated_reward"]
        .mean()
        .reset_index()
        .rename(columns={"aggregated_reward": "mean_reward"})
    )

    # Apply moving average
    episode_avg["moving_avg"] = (
        episode_avg["mean_reward"].rolling(window=window_size, min_periods=1, center=True).mean()
    )

    if plot:
        plt.figure(figsize=(10, 5))
        plt.plot(episode_avg[episode_column], episode_avg["moving_avg"], label=f"Moving Avg ({aggregation_mode})")
        plt.xlabel("Episode")
        plt.ylabel("Reward")
        plt.title(f"{aggregation_mode.capitalize()} Agent Reward (Window={window_size})")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

    return episode_avg


### Variable_calculation

#### Gini coefficient

In [12]:
def gini_coefficient(arr: np.ndarray) -> float:
    """Compute Gini coefficient of a 1D numpy array."""
    arr = arr.flatten()
    if np.amin(arr) < 0:
        arr = arr - np.amin(arr)  # Shift if negative values present
    mean = np.mean(arr)
    if mean == 0:
        return 0.0
    n = len(arr)
    diff_sum = np.sum(np.abs(np.subtract.outer(arr, arr)))
    gini = diff_sum / (2 * n**2 * mean)
    return gini

def compute_gini_for_df(df: pd.DataFrame, prefix: str) -> pd.Series:
    """
    Compute Gini coefficient across columns starting with prefix for each row in df.
    
    Args:
        df: pandas DataFrame.
        prefix: string prefix for target columns.
        
    Returns:
        pandas Series with Gini coefficients per row.
    """
    # Select columns matching the prefix
    cols = [col for col in df.columns if col.startswith(prefix)]
    if not cols:
        raise ValueError(f"No columns found starting with prefix '{prefix}'")

    # Apply gini_coefficient row-wise across selected columns
    gini_series = df[cols].apply(lambda row: gini_coefficient(row.values), axis=1)
    return gini_series

#### Ressource depletion : to do

# Analysis of data

In [ ]:
# processing of filenames to get the parameters' values tested
data_folder_name_acces = ""
files_values = parse_results_filenames(data_folder_name_acces)

# agregation of 2D Grid data
df_maze_summary = aggregate_results_by_suffix(data_folder_name_acces, "episode_summary", source_filter="maze2d")
df_maze_step = aggregate_results_by_suffix(data_folder_name_acces, "step_data", source_filter="maze2d")

# Only aggregate GameTheoretic step data
df_gt_summary = aggregate_results_by_suffix(data_folder_name_acces, "step_data", source_filter="Gametheoretic")
df_gt_step = aggregate_results_by_suffix(data_folder_name_acces, "step_data", source_filter="Gametheoretic")

## Calculation of Dependent Variables

In [ ]:
# Computes the gini coef over personnal rewards
df_maze_summary["gini_personal_reward"] = compute_gini_for_df(df_maze_summary, prefix="total_personal_reward_")
df_maze_step["gini_personal_reward"] = compute_gini_for_df(df_maze_step, prefix="total_personal_reward_")

df_gt_summary["gini_personal_reward"] = compute_gini_for_df(df_gt_summary, prefix="personal_")
df_gt_step["gini_personal_reward"] = compute_gini_for_df(df_gt_step, prefix="personal_")

## Learning verification

In [ ]:
df = df_maze_summary # summary csv to test for learning

windowed_df = windowed_avg_combined_reward(
    df=df,                                # summary dataframe to calculate upon
    reward_prefix="total_combined_reward_",  # or "personal_"
    episode_column="episode",             # the name of your episode column
    simulation_id_column="simulation_index",  # the name of your simulation ID column
    window_size=10,                       # adjust smoothing window size
    aggregation_mode="mean",              # "mean" for average agent, "best" for best agent
    plot=True                             # show plot of the result
)

## Data Vizualization

## Data Analysis

# Interpretation